## Practice Calling Data_Gathering_Functions

In [7]:
# import the data_gathering_function module to have access to all cleaning and gathering materials
from data_gathering_function import *

In [8]:
# spotify's api access token needs to be refreshed every hour. 
# by executing this cell, the access token is refreshed
get_tokens()

Full Token Data
{'access_token': 'BQCBsrXERhJwVhyQHsmt5eVDw2h3z7AAFcGcta0N35tMEWGD_709E49HMfuBFuBKhO0roxScz1jXk5glcSY', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}


token needed: BQCBsrXERhJwVhyQHsmt5eVDw2h3z7AAFcGcta0N35tMEWGD_709E49HMfuBFuBKhO0roxScz1jXk5glcSY


{'access_token': 'BQCBsrXERhJwVhyQHsmt5eVDw2h3z7AAFcGcta0N35tMEWGD_709E49HMfuBFuBKhO0roxScz1jXk5glcSY',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'scope': ''}

In [5]:
# dictionary of genre and respective playlist ID
genre_dict

{'hip hop playlist': '7Mr3wEdKgaiAP4Cm2a6vda',
 'post rock playlist': '4MRGjKqlWuJZJ8XHOGcqkR',
 'electronic playlist': '37i9dQZF1DX4dyzvuaRJ0n',
 'detroit_techo playlist': '1o1HVRGIxwCcdSnNnZ69IC',
 'classical playlist': '37i9dQZF1DWWEJlAGA9gs0',
 'disco playlist': '0TCtFMz5lY6jTfusk66ZFj',
 'electro indie pop playlist': '3kTtdRE1CtRyRKdicfOGAR',
 'industrial pop playlist': '30BUPgw52SWNm2ZWZZc86A',
 'french playlist': '6Ph1K0QWCcEwYRr0VhVt6C',
 'spanish playlist': '5khoF3ksobwfVwOazDqpqI',
 'ska playlist': '6mRRGF4klfgUzbD2ZKOCq0',
 'sleep playlist': '37i9dQZF1DWZd79rJ6a7lp',
 'rockabilly playlist': '37i9dQZF1DX0xLQsW8b5Zx',
 '50s Hits playlist': '2SwjQPegrTTYaOsWQrwhMe'}

In [9]:
# how to retrieve genre high level analysis dataframe
classical_df = call_spotify_return_feat_df(genre_dict.get('classical playlist'))
classical_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.2210,0.1260,0,-25.427,1,0.0447,0.989,0.897,0.1020,0.2160,133.630,audio_features,4SFBV7SRNG2e2kyL1F6kjU,spotify:track:4SFBV7SRNG2e2kyL1F6kjU,https://api.spotify.com/v1/tracks/4SFBV7SRNG2e...,https://api.spotify.com/v1/audio-analysis/4SFB...,139307,4
1,0.0811,0.0122,4,-32.654,0,0.0511,0.902,0.308,0.0648,0.0384,74.554,audio_features,2kAgCRZPG3YQR2VMqRvLmb,spotify:track:2kAgCRZPG3YQR2VMqRvLmb,https://api.spotify.com/v1/tracks/2kAgCRZPG3YQ...,https://api.spotify.com/v1/audio-analysis/2kAg...,935360,4
2,0.2040,0.0331,2,-24.044,0,0.0487,0.927,0.825,0.0954,0.0547,116.200,audio_features,1upQiytDIEZfl9ItruoXuC,spotify:track:1upQiytDIEZfl9ItruoXuC,https://api.spotify.com/v1/tracks/1upQiytDIEZf...,https://api.spotify.com/v1/audio-analysis/1upQ...,207267,3
3,0.2300,0.0487,5,-27.675,1,0.0412,0.986,0.945,0.0679,0.1010,134.839,audio_features,0H2VhGUC3P3hvwz8rdGIpC,spotify:track:0H2VhGUC3P3hvwz8rdGIpC,https://api.spotify.com/v1/tracks/0H2VhGUC3P3h...,https://api.spotify.com/v1/audio-analysis/0H2V...,203656,5
4,0.2050,0.0525,1,-23.841,1,0.0327,0.980,0.906,0.1030,0.0511,88.316,audio_features,419qlOGNXoyICx4jO0hqaj,spotify:track:419qlOGNXoyICx4jO0hqaj,https://api.spotify.com/v1/tracks/419qlOGNXoyI...,https://api.spotify.com/v1/audio-analysis/419q...,174653,4
